In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DATA_LOC = "/content/drive/MyDrive/Rhexis/datasets/test_pulls"
REPO_LOC = "/content/drive/MyDrive/Projects/rhexis-trajectory"

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
sys.path.insert(0,f"{REPO_LOC}/Trajectory_Classification")
from utils import *
%load_ext autoreload
%autoreload 2

# Load data

In [ ]:
names, path_dfs, labels, sizes = load_all_pulls(DATA_LOC)

In [ ]:
path_dfs[0]

In [ ]:
X, y = np.stack([featurize_pull(pull, 15) for pull in path_dfs], axis=0), np.array(labels)
X[0]

In [ ]:
X_train, X_test, y_train, y_test = stratified_split_data(X, y)

# Print to check class balance
y_train, y_test

In [ ]:
# NUM_ANGLE_BINS = list(range(3, 30))
# for num_bins in NUM_ANGLE_BINS:

  # X, y = np.stack([featurize_pull(pull, num_bins) for pull in path_dfs], axis=0), np.array(labels)
  # np.random.seed(13)
  # sss = StratifiedShuffleSplit(1, test_size=.2)
  # train_ind, test_ind = next(sss.split(X, y))
  # X_train, X_test = X[train_ind], X[test_ind]
  # y_train, y_test = y[train_ind], y[test_ind]
  # clf = LogisticRegression(random_state=0, multi_class="multinomial", max_iter=1000)
  # pipe = make_pipeline(StandardScaler(), clf)
  # print(cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy'))

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, max_iter=1000)
param_grid = {
    'pca__n_components': list(range(3, 20)),
}

search = GridSearchCV(make_custom_pipeline(clf, include_pca=True), param_grid, cv=5)
search.fit(X_train, y_train)
search.best_params_
search.best_score_

In [ ]:
param_grid = {
    'pca__n_components': list(range(3, 20)),
}
bin_grid = list(range(5, 25))
results = grid_search_with_bins(make_custom_pipeline(clf, True), param_grid, bin_grid)

In [ ]:
results["best_score"]

In [ ]:
results["best_search"].score(results["X_test"], results["y_test"])

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

clf = QuadraticDiscriminantAnalysis()
pipe = make_pipeline(StandardScaler(), clf)
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy')

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import accuracy_score

BATCH_SIZE = 3
kmeans = MiniBatchKMeans(n_clusters=3,
                          random_state=0,
                          batch_size=BATCH_SIZE)

pipe = make_pipeline(StandardScaler(), kmeans)
for i in range(0, len(X_train), BATCH_SIZE):
  pipe.partial_fit(X_train[i:i+BATCH_SIZE])

y_pred = kmeans.predict(X_test)
# accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(50, 25), random_state=1)
pipe = make_pipeline(StandardScaler(),
                     clf)
pipe.fit(X_train, y_train)
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy')
